In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
import matplotlib.pyplot as plt
import librosa
import librosa.display

import warnings
warnings.filterwarnings('ignore')

In [2]:
speech_reader = pd.read_csv("C:/Python/PlayArea/speech-emotion-recognition-ravdess-data.csv", delimiter=',')

In [4]:
extracted_features=[]

for index_num,row in speech_reader.iterrows():
    #print(row)
    emotion = row['Emotions']
    observed_emotions=['sad', 'happy', 'fearful', 'disgust','angry','surprised','calm']
    #observed_emotions=['sad']
    if emotion not in observed_emotions:
            continue
    feature_result=np.array([])
    file_name = row['File_Path'] + '/' + row['File_Name']
    #print(file_name)
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    #we extract mfcc
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    feature_result = np.hstack((feature_result, mfccs_scaled_features))
    #we extract chroma
    stft=np.abs(librosa.stft(audio))
    chroma_features = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    feature_result = np.hstack((feature_result, chroma_features))
    #we extract mel
    mel_features = np.mean(librosa.feature.melspectrogram(audio, sr=sample_rate).T,axis=0)
    #mel_features1.append(mel_features)
    feature_result = np.hstack((feature_result, mel_features))
    feature_result = feature_result.tolist()
    lable = row['Gender'] + '_' + row['Emotions']
    extracted_features.append([feature_result,lable])

In [ ]:
#extracted_features_df1=pd.DataFrame(extracted_features,columns=['feature','class_lable'])
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class_lable'])

In [ ]:
extracted_features_df

In [ ]:
extracted_features_df.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
### Split the dataset into independent and dependent dataset
X = np.asarray(extracted_features_df['feature'].tolist())
y1 = np.asarray(extracted_features_df['class_lable'].tolist())

In [ ]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=train_test_split(X,y1,test_size=0.2,random_state=0)

In [ ]:
print({x_train.shape[0] , x_test.shape[0], x_train.shape[0] + x_test.shape[0] })

In [ ]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

In [ ]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#DataFlair - Train the model
model.fit(x_train,y_train)

In [ ]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Create the confusion matrix
confusion_matrix1 = confusion_matrix(y_test, y_pred)
#confusion_matrix(y_test, y_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix1, display_labels=model.classes_)
#disp.plot()

In [ ]:
from sklearn.utils.multiclass import unique_labels
df_cm = pd.DataFrame(confusion_matrix1, index = unique_labels(y_test, y_pred), columns = unique_labels(y_test, y_pred))
fig, ax = plt.subplots(1, 1, figsize=(10,7))
heatmap = sns.heatmap(df_cm, annot=True, ax=ax, fmt="d")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
preds = model.predict(x_test)
preds = pd.DataFrame({'predictedvalues': preds})
preds


In [ ]:
actual=y_test
actual = pd.DataFrame({'actualvalues': actual})

In [ ]:
finaldf = actual.join(preds)

In [ ]:
finaldf[170:180]

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Classification report 
classes = finaldf.actualvalues.unique()
classes.sort()    
print(classification_report(finaldf.actualvalues, finaldf.predictedvalues, target_names=classes))